In [80]:
#Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

from data_manager import get_data


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures


from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV


In [81]:
ds2, ds3 = get_data()
print(ds3)
print(len(ds3[ds3['target'] == 1]))

        TP    DP    Cl    TN      TempC   Chla  Secchi   NP_Cya_bio  target  \
0     39.2  16.2  13.0  0.61   6.494521   1.41     0.5          0.0     0.0   
1     36.8  14.8  17.5  0.45  13.700000   9.67     1.1          0.0     0.0   
2     50.1  27.4  12.1  0.55  14.500000   2.04     0.7          0.0     0.0   
4     59.6  32.6  12.0  0.65  17.700000   4.13     0.6          0.0     0.0   
5     77.3  47.9  10.5  0.62  22.500000   1.74     0.6          0.0     0.0   
...    ...   ...   ...   ...        ...    ...     ...          ...     ...   
3629  53.4  16.8   8.0  0.69  25.600000  27.50     1.1  389000000.0     0.0   
3631  83.4  33.9   8.3  0.71  23.700000  23.94     1.0  133000000.0     0.0   
3632  94.2  40.7   8.7  0.90  22.300000  50.16     1.0  443000000.0     1.0   
3634  68.8  42.6   9.6  0.74  13.400000  10.22     1.4    9460000.0     0.0   
3636  79.4  49.7   9.2  0.76   9.200000  11.81     1.3    6510000.0     0.0   

            N:P  Month  
0     34.410892      4  
1

In [82]:
#Create X and y
X = np.array(ds3.drop(['target', 'NP_Cya_bio'], axis=1))
y = np.array(ds3['target'])
y_reg = np.array(ds3['NP_Cya_bio']) #for regression


In [83]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X, y_reg, test_size=0.20, random_state = 42)

In [84]:
#Scale the X's for logistic regression (didn't help for linear regression)
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [85]:
#Try adding polynomial features

#Define Polynomial transformation 
poly = PolynomialFeatures(2, include_bias=False)

#Map the features
poly_X_train = poly.fit_transform(X_train)
poly_X_test = poly.fit_transform(X_test)

poly_Xr_train = poly.fit_transform(X_reg_train)
poly_Xr_test = poly.fit_transform(X_reg_test)
#print(poly_X_train.shape, poly_X_test.shape)


In [86]:
#Linear regression
model = linear_model.LinearRegression()
model.fit(X_reg_train, y_reg_train)
#Testing
print(model.score(X_reg_test, y_reg_test))
print(model.coef_) # get theta coefficients (model params)

#CV - doesn't really work, too much variation
#r2 = cross_val_score(model, X, y_reg, scoring = 'r2', cv = 5, )


#Linear regression w/ polynomial features
model = linear_model.LinearRegression()
model.fit(poly_Xr_train, y_reg_train)
#Testing
print(model.score(poly_Xr_test, y_reg_test))
print(model.coef_) # get theta coefficients (model params)

0.2966766257913154
[  126475.2968059   -517697.54255332 -1304943.14617989 32969247.88086407
  -205743.17908729 12238131.67368069 11312522.94010345   217084.49920213
  1908673.49844075]
0.42085912012181614
[-4.33607466e+05  1.97232132e+06  1.24166427e+07 -6.65851496e+08
 -1.06236033e+07  8.00611714e+06 -7.69035755e+06  1.54813859e+06
  1.97225206e+07 -1.38367122e+04  2.42157478e+05 -1.97075980e+05
 -1.86005015e+06 -9.41496094e+04 -2.64407251e+05 -4.87883686e+05
 -3.47452373e+03  7.40893805e+05 -1.66456211e+05  2.48169153e+05
 -6.25516534e+06  3.53042200e+05 -4.44816625e+05  3.18073790e+05
  5.16334387e+04 -1.59704180e+06 -2.69593773e+04  2.47966742e+05
  7.83502296e+04 -6.26347207e+05 -1.20386100e+06 -4.11771916e+03
 -6.16001196e+05  3.21433926e+08 -1.12271961e+07  4.74361463e+07
  9.55392088e+07 -2.87281592e+06  4.32759541e+07 -1.36609238e+05
  7.75879891e+05  4.91221837e+05  7.94201506e+04  8.88886103e+05
  2.11625611e+04 -1.30850724e+06 -2.86074181e+05 -7.66776322e+05
  4.20282550e+0

In [87]:
#Try again using other method? Uses all data, gives p-values give feature significance!
import statsmodels.api as sm
# Fit regression model
model = sm.OLS(y_reg,X)
result = model.fit()
print(result.summary2())

#And with polynomial features:
model = sm.OLS(y_reg_train, poly_Xr_train)
result = model.fit()
print(result.summary2())

                        Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.440     
Dependent Variable:     y                AIC:                         56672.0654
Date:                   2020-05-07 13:56 BIC:                         56719.4605
No. Observations:       1431             Log-Likelihood:              -28327.   
Df Model:               9                F-statistic:                 125.7     
Df Residuals:           1422             Prob (F-statistic):          8.98e-174 
R-squared (uncentered): 0.443            Scale:                       9.2091e+15
---------------------------------------------------------------------------------
         Coef.         Std.Err.       t     P>|t|       [0.025          0.975]   
---------------------------------------------------------------------------------
x1      36588.6634    429892.5307   0.0851  0.9322    -806702.9886    879880.3155
x2    -962534.7944    660042.2441  -1.4583  0.145

In [88]:
#Basic logistic Regression
model = LogisticRegression(solver = "liblinear", penalty = 'l1', C = 1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

recall = metrics.recall_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred)
print('Recall:', recall)
print('ROC AUC:', auc)

scores = metrics.classification_report(y_test, y_pred)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(scores)
print(confusion_matrix)


Recall: 0.25
ROC AUC: 0.6214664310954063
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       283
         1.0       0.33      0.25      0.29         4

    accuracy                           0.98       287
   macro avg       0.66      0.62      0.64       287
weighted avg       0.98      0.98      0.98       287

[[281   2]
 [  3   1]]


In [89]:
#Basic Logistic regression plus polynomial features:
model = LogisticRegression(solver = "liblinear", penalty = 'l1', C = 1)
model.fit(poly_X_train, y_train)

y_pred = model.predict(poly_X_test)

recall = metrics.recall_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred)
print('Recall:', recall)
print('ROC AUC:', auc)

scores = metrics.classification_report(y_test, y_pred)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(scores)
print(confusion_matrix)


Recall: 0.5
ROC AUC: 0.7464664310954063
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       283
         1.0       0.50      0.50      0.50         4

    accuracy                           0.99       287
   macro avg       0.75      0.75      0.75       287
weighted avg       0.99      0.99      0.99       287

[[281   2]
 [  2   2]]


In [106]:
#Basic logistic regression - tune hyperparams to find the best model! 

#Ridge
model = LogisticRegression(solver = "liblinear", max_iter = 10000, penalty = 'l2')

#Use CV to find best parameters: 
best_estimators = []
distros = dict(C = np.logspace(-4, 2, 7), 
               class_weight = ['balanced', None])

search = RandomizedSearchCV(model, distros, scoring='roc_auc', refit='AUC', verbose=5, cv=5, n_iter=800, n_jobs=4, pre_dispatch='2*n_jobs')
search = search.fit(X_train, y_train)
best_estimators.append(search.best_estimator_)
best_estimators.append(search.best_score_)

#Lasso
model = LogisticRegression(solver = "liblinear", max_iter = 10000, penalty = 'l1')
distros = dict(C = np.logspace(-4, 2, 7), 
               class_weight = ['balanced', None])

search = RandomizedSearchCV(model, distros, scoring='roc_auc', refit='AUC', verbose=5, cv=5, n_iter=800, n_jobs=4, pre_dispatch='2*n_jobs')
search = search.fit(X_train, y_train)
best_estimators.append(search.best_estimator_)
best_estimators.append(search.best_score_)

#Elasticnet
model = LogisticRegression(solver = "saga", penalty = 'elasticnet', max_iter = 10000)
distros = dict(C = np.logspace(-4, 2, 7), 
               class_weight = ['balanced', None],
              l1_ratio = [.1, .5, .7, .9, .95, .99, 1])
search = RandomizedSearchCV(model, distros, scoring='roc_auc', refit='AUC', verbose=5, cv=5, n_iter=800, n_jobs=4, pre_dispatch='2*n_jobs')
search = search.fit(X_train, y_train)
best_estimators.append(search.best_estimator_)
best_estimators.append(search.best_score_)
best_estimators

Fitting 5 folds for each of 14 candidates, totalling 70 fits


/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 14 is smaller than n_iter=800. Running 14 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:    3.1s finished
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 14 is smaller than n_iter=800. Running 14 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:    0.5s finished
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 98 is smaller than n_iter=800. Running 98 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 98 candidates, totalling 490 fits


[Parallel(n_jobs=4)]: Done  60 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 490 out of 490 | elapsed:  2.1min finished
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=10000, multi_class='warn', n_jobs=None,
                    penalty='l2', random_state=None, solver='liblinear',
                    tol=0.0001, verbose=0, warm_start=False),
 0.9636835595574533,
 LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=10000,
                    multi_class='warn', n_jobs=None, penalty='l1',
                    random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                    warm_start=False),
 0.9540243139358185,
 LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=0.1, max_iter=10000,
                    multi_class='warn', n_jobs=None, penalty='elasticnet',
                    random_state=None, solver='sa

In [112]:
# Logistic Regression with polynomial features - tune hyperparams to find the best model

#Ridge
model = LogisticRegression(solver = "liblinear", max_iter = 10000, penalty = 'l2')

#Use CV to find best parameters: 
#best_estimators = []
distros = dict(C = np.logspace(-4, 2, 7), 
               class_weight = ['balanced', None])

search = RandomizedSearchCV(model, distros, scoring='roc_auc', refit='AUC', verbose=5, cv=5, n_iter=800, n_jobs=4, pre_dispatch='2*n_jobs')
search = search.fit(poly_X_train, y_train)
best_estimators.append(search.best_estimator_)
best_estimators.append(search.best_score_)

#Lasso
model = LogisticRegression(solver = "liblinear", max_iter = 10000, penalty = 'l1')
distros = dict(C = np.logspace(-4, 2, 7), 
               class_weight = ['balanced', None])

search = RandomizedSearchCV(model, distros, scoring='roc_auc', refit='AUC', verbose=5, cv=5, n_iter=800, n_jobs=4, pre_dispatch='2*n_jobs')
search = search.fit(poly_X_train, y_train)
best_estimators.append(search.best_estimator_)
best_estimators.append(search.best_score_)

#Elasticnet
model = LogisticRegression(solver = "saga", penalty = 'elasticnet', max_iter = 10000)
distros = dict(C = np.logspace(-4, 2, 7), 
               class_weight = ['balanced', None],
              l1_ratio = [.1, .5, .7, .9, .95, .99, 1])
search = RandomizedSearchCV(model, distros, scoring='roc_auc', refit='AUC', verbose=5, cv=5, n_iter=800, n_jobs=4, pre_dispatch='2*n_jobs')
search = search.fit(poly_X_train, y_train)
best_estimators.append(search.best_estimator_)
best_estimators.append(search.best_score_)
best_estimators

Fitting 5 folds for each of 14 candidates, totalling 70 fits


/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 14 is smaller than n_iter=800. Running 14 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:    3.2s finished
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 14 is smaller than n_iter=800. R

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:    7.4s finished
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 98 is smaller than n_iter=800. Running 98 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 98 candidates, totalling 490 fits


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 265 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 372 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done 490 out of 490 | elapsed:  8.3min finished
/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=10000, multi_class='warn', n_jobs=None,
                    penalty='l2', random_state=None, solver='liblinear',
                    tol=0.0001, verbose=0, warm_start=False),
 0.9636835595574533,
 LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=10000,
                    multi_class='warn', n_jobs=None, penalty='l1',
                    random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                    warm_start=False),
 0.9540243139358185,
 LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=0.1, max_iter=10000,
                    multi_class='warn', n_jobs=None, penalty='elasticnet',
                    random_state=None, solver='sa

In [118]:
#Model 1 with Ridge (L2) does the best without polynomial features
model = LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=10000, multi_class='warn', n_jobs=None,
                    penalty='l2', random_state=None, solver='liblinear',
                    tol=0.0001, verbose=0, warm_start=False)
model2 = LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=10000,
                    multi_class='warn', n_jobs=None, penalty='l1',
                    random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                    warm_start=False)
model3 = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=0.1, max_iter=10000,
                    multi_class='warn', n_jobs=None, penalty='elasticnet',
                    random_state=None, solver='saga', tol=0.0001, verbose=0,
                    warm_start=False)

model1.fit(X_train, y_train)

#Testing
y_pred = model1.predict(X_test)
recall = metrics.recall_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred)
print('Recall:', recall)
print('ROC AUC:', auc)
scores = metrics.classification_report(y_test, y_pred)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(scores)
print(confusion_matrix)

#Results: Recall, ROC AUC for
#model1: 1, 0.86 
#model2: 0.25, 0.62
#model3: 0.25, 0.62

Recall: 1.0
ROC AUC: 0.8621908127208481
              precision    recall  f1-score   support

         0.0       1.00      0.72      0.84       283
         1.0       0.05      1.00      0.09         4

    accuracy                           0.73       287
   macro avg       0.52      0.86      0.47       287
weighted avg       0.99      0.73      0.83       287

[[205  78]
 [  0   4]]


In [116]:
#Models with polynomial features:
#Mixed results depending whether we care more about recall or ROC AUC
model1 = LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=10000, multi_class='warn', n_jobs=None,
                    penalty='l2', random_state=None, solver='liblinear',
                    tol=0.0001, verbose=0, warm_start=False)
model2 = LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=10000, multi_class='warn', n_jobs=None,
                    penalty='l1', random_state=None, solver='liblinear',
                    tol=0.0001, verbose=0, warm_start=False)
model3 = LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=0.7,
                    max_iter=10000, multi_class='warn', n_jobs=None,
                    penalty='elasticnet', random_state=None, solver='saga',
                    tol=0.0001, verbose=0, warm_start=False)

model2.fit(poly_X_train, y_train)

#Testing
y_pred = model2.predict(poly_X_test)
recall = metrics.recall_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred)
print('Recall:', recall)
print('ROC AUC:', auc)
scores = metrics.classification_report(y_test, y_pred)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(scores)
print(confusion_matrix)

#Results: Recall, ROC AUC for
#model1: 0.75, 0.78
#model2: 1, 0.5
#model3: 0.5, 0.74

Recall: 1.0
ROC AUC: 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       283
         1.0       0.01      1.00      0.03         4

    accuracy                           0.01       287
   macro avg       0.01      0.50      0.01       287
weighted avg       0.00      0.01      0.00       287

[[  0 283]
 [  0   4]]


/Users/mahaliaclark/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Things to try for logistic regression:
- ds2 vs ds3
- features scaled vs not
- use cross validation
- get performance summary with recall and ROC AUC and confusion matrix
- **add polynomial features**
- add regularization: try elastic net and/or Lasso (=L1?)
- use random search and cross validation to tune hyperparameters, find best model (specify scoring metric as recall/F1/AUC?):
    - C (inverse of regularization strength --> smaller value = more regularization)
    - solver (choice depends on choice of regularization)
    - penalty (form of regularization?)
    - l1_ratio: ratio between 0 and 1 passed for elasticnet